In [1]:
import sys
import json
import os
import requests
import random
from datetime import datetime
import time

import pandas as pd
import numpy as np
import sklearn.linear_model




## Run a serving workload

This just runs a light prediction-serving workload and prints the accuracy after every 100 predictions

In [2]:
def load_digits(digits_location, digits_filename = "train.data", norm=True):
    digits_path = digits_location + "/" + digits_filename
    print("Source file: %s" % digits_path)
    df = pd.read_csv(digits_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    label = 3
    my_y = [1. if i == label else -1.0 for i in y]
    return (Z, my_y)


def mnist_update(host, uid, x, y):
    url = "http://%s:1337/update" % host
    req_json = json.dumps({'uid': uid, 'input': list(x), 'label': y})
    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

def mnist_prediction(host, uid, x):
    url = "http://%s:1337/predict" % host
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    # x_str = ", ".join(["%d" % a for a in x])
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    pred = float(r.text.split(":")[1].strip())
    if pred == 0:
        pred = -1.0
    return (pred, latency)



In [ ]:

host = "ec2-54-158-88-117.compute-1.amazonaws.com"
x, y = load_digits(os.path.expanduser("~/model-serving/data/mnist_data"), digits_filename = "test.data")
# z = normalize_digits(x)
# inputs = args[2:]
uid = 0
# num_inputs = int(args[2])
iii = 0
while True
    correct = 0
    false_pos = 0
    false_neg = 0
    latencies = []
    for i in range(100):
        iii += 1
        # mnist_update(uid, x[int(i)], float(y[int(i)]))
        example_num = np.random.randint(0,len(x))
        correct_y = float(y[example_num])
        # mnist_update(uid, x[example_num], float(y[example_num]))
        pred_y, latency = mnist_prediction(host, uid, x[example_num][:3])
        if correct_y == pred_y:
            correct += 1
        elif correct_y == -1 and pred_y == 1:
            false_pos += 1
        elif correct_y == 1 and pred_y == -1:
            false_neg += 1
        else:
            print "predicted: {p}, correct: {c}".format(p=pred_y, c=correct_y)
        latencies.append(latency)
        time.sleep(10)
    print "Accuracy: {cor}, false positives: {fp}, false negatives: {fn}".format(
        cor = float(correct)/100.0,
        fp=float(false_pos)/100.0,
        fn=float(false_neg)/100.0)
            
    


In [3]:

host = "ec2-54-158-88-117.compute-1.amazonaws.com"
x, y = load_digits(os.path.expanduser("~/model-serving/data/mnist_data"), digits_filename = "test.data")
# z = normalize_digits(x)
# inputs = args[2:]
uid = 0
example_num = np.random.randint(0,len(x))
correct_y = float(y[example_num])
# mnist_update(uid, x[example_num], float(y[example_num]))
pred_y, latency = mnist_prediction(host, uid, x[example_num])
# if correct_y == pred_y:
#     correct += 1
# elif correct_y == -1 and pred_y == 1:
#     false_pos += 1
# elif correct_y == 1 and pred_y == -1:
#     false_neg += 1
# else:
print "predicted: {p}, correct: {c}".format(p=pred_y, c=correct_y)

Source file: /Users/crankshaw/model-serving/data/mnist_data/test.data
Number of image files: 10000


ConnectionError: ('Connection aborted.', error(54, 'Connection reset by peer'))

In [ ]:
time.sleep(4.5)

In [ ]:
p = np.array([-1, 1, 1, 1, -1])
c = np.array([1, 1, 1, -1, -1])

In [ ]:
np.sum(c - p)